# overview
* basic python
* accessing data
* storing data in database
* manipulating dataframes
* visualizing data
* split-apply-combine
* heatmaps
* simple multilable classifier

## basic python

In [ ]:
# import some libraries
import pandas as pd # dataframes
import seaborn as sns # visualization
%pylab inline

In [ ]:
# explain variables and dynamic types in python
a=1
print(a)
a="eins"
print(a)

In [ ]:
# explain list / set
mylist = [1,1,2,2,3,4,5,'sechs','sieben',0x08]
print(mylist)
myset = set(mylist)
print(myset)

In [ ]:
# explain dictionary
mydict = dict()
mydict['Andreas Steffen']=1
mydict['Djork-Arne']=5
print(mydict)
mydict = {'Andreas Steffen': 1, 'Djork-Arne': 5}
print(mydict)

In [ ]:
# show a for loop
for ele in mylist:
    print(ele)

In [ ]:
# show list comprehension
[ele for ele in mylist if isinstance(ele, int)]

In [ ]:
# import the random lib
import random

In [ ]:
# a simple function
def random_int(max_int):
    ret_val = random.randint(0, max_int)
    return ret_val

In [ ]:
random_int(99)

In [ ]:
# explain dataframes
df = pd.DataFrame([random_int(100) for _ in range(10)])
print(df)
df = pd.DataFrame(columns = ['col1','col2'], data = [[1,'eins'],[2,'zwei']])
print(df)

In [ ]:
print(df.dtypes)
df.describe()

# enough basics, let's do some data science

In [ ]:
# read in the data expression. it's in the data folder
data = pd.read_table('../data/expression.tsv')

In [ ]:
# let's inspect
data.head()

In [ ]:
# write data into a sqlite db
import sqlite3 as lite
con = lite.connect('../data/expression.db')
data.to_sql('expression',if_exists = 'replace', index = None, con = con)

In [ ]:
# and read it back
data1 = pd.read_sql('SELECT * FROM expression', con)

In [ ]:
data1.head()

In [ ]:
# show indexing
data.loc[[1,2],:]

In [ ]:
data.loc[[1,2], ['title','plot']]

In [ ]:
# only selected columns
data[['title','plot']]

In [ ]:
# let's extract the year
data['year'] = data.title.str.extract('\((\d{4})', expand = True)[0]
#data['year'] = data.title.str.extract('\((\d{4})(/[IVXD]{1,3})?\)', expand = True)[0]

In [ ]:
data.year.unique()

In [ ]:
# find NA years
data[pd.isnull(data.year)]

In [ ]:
# drop the ones without a year
data = data.dropna(subset = ['year'])

In [ ]:
# drop duplicates by year and plot
data = data.drop_duplicates(['year', 'plot'])

In [ ]:
# plot by year
plt.figure(figsize = (20,9))
data.year.value_counts().sort_index().plot.bar()
sns.despine()

In [ ]:
!pip3 install langdetect

In [ ]:
# only english
from langdetect import detect

In [ ]:
print(detect("My name is Andreas Steffen"))

In [ ]:
print(detect("Ich heisse Andreas Steffen"))

In [ ]:
# show for top 100
[detect(plot) for plot in data['plot'].head(100).tolist()]

In [ ]:
# do for all takes to long
data['language'] = [detect(plot) for plot in data['plot'].tolist()]

In [ ]:
# read in language 
#data.to_csv('../data/movies_genres.with_year_lang.tsv', sep = '\t')
data = pd.read_table('../data/movies_genres.with_year_lang.tsv', sep = '\t',index_col = 0)
#data.shape

In [ ]:
# show stats
data.language.value_counts()

In [ ]:
# check some languages
data.query("language == 'it'")['plot'].tolist()

In [ ]:
data = data.query("language == 'en'")

In [ ]:
# get genres
genres = data.filter(regex = "[A-Z]", axis = 1).columns.tolist()

In [ ]:
genres

In [ ]:
# plot genre occurence
plt.figure(figsize = (20,9))
data[genres].sum().plot.bar()
sns.despine()

In [ ]:
# melt the data
data_melt = pd.melt(data,id_vars = ['title', 'plot'], value_vars = genres, var_name = 'genre')

In [ ]:
# remove zeros
data_melt = data_melt[data_melt.value!=0]

In [ ]:
# calculate average number of chars in plot description per genre and plot
plt.figure(figsize = (20,9))
data_melt.groupby('genre').apply(lambda x: x['plot'].str.len().mean()).plot.bar()
sns.despine()

In [ ]:
# calculate the plot string len
data_melt['plot_string_len'] = data_melt['plot'].str.len()

In [ ]:
# but let's use a boxplot
plt.figure(figsize=(20,9))
g = sns.boxplot(data = data_melt, x= 'genre', y= 'plot_string_len', color = '#dd00aa')
g.set( yscale="log")
plt.xticks(rotation = 90)
sns.despine()

In [ ]:
# inspect strongst outlier
data_melt.sort_values('plot_string_len', ascending = False).head(1)['plot'].values

In [ ]:
# calculate similarity of genres
from scipy.spatial import distance as d

In [ ]:
# filter the genre subset
data_genres = data.filter(regex = '[A-Z]', axis = 1)

In [ ]:
data_genres.head()

In [ ]:
# plot the similarity heatmap
res = 1-d.squareform(d.pdist(data_genres.T, 'jaccard'))
plt.figure(figsize = (10,10))
sims = pd.DataFrame(res, index=data_genres.columns, columns= data_genres.columns)
sns.heatmap(sims)

# let's do some descriptive stats, man!

In [ ]:
# ttest

In [1]:
# xkcd multiple testing

In [2]:
# multiple testing

In [ ]:
# correlation

# how about training a predictive biomarker?

In [ ]:
# import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVCxm

In [ ]:
# transformation and training pipeline
tfidf = TfidfVectorizer()
classifier = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(RandomForestClassifier()))])

In [ ]:
# just train on the top 10000 in the interest of time
train_x = data['plot'].head(1000)

In [ ]:
# convert to a matrix format (just because sklearn needs it)
train_y = data_genres.head(1000).as_matrix()

In [ ]:
# fit the model
classifier.fit(train_x, train_y)

In [ ]:
# let's fetch some movie descriptions from itunes
actor = 'Cate Blanchett'.replace(' ','+')
test = pd.read_json('https://itunes.apple.com/search?term={actor}&entity=movie&limit=200'.format(actor = actor))

In [ ]:
# some tweaking of the input
test_extract = test.results.apply(lambda x: pd.Series({'plot': x['longDescription'],'title':x['trackName']}) )

In [ ]:
test_extract

In [ ]:
# predict probabilities
predicted = classifier.predict_proba(test_extract['plot'])

In [ ]:
# convert to dataframe
predicted = pd.DataFrame(predicted, columns = data_genres.columns, index = test_extract.title)

In [ ]:
# and plot
plt.figure(figsize=(20,20))
sns.heatmap(predicted)

In [ ]:
from IPython.display import Javascript
#runs arbitrary javascript, client-side
Javascript("""
           window.datatable={};
           console.log('test');
           """.format(predicted.reset_index().to_json(orient='records')))

In [ ]:
predicted.reset_index()

In [ ]:
%%javascript
element.append("<div id = 'lineuptable'></div>")
require(['./LineUpJS_bundle.min.js'],function(LineUpJS){
    const arr = window.datatable
    console.log(arr);
    const desc = [
              {label: 'title', type: 'string', column: 'title'},
              {label: 'Drama', type: 'number', column: 'Drama', 'domain': [0, 1]},
              {label: 'Romance', type: 'number', column: 'Romance', 'domain': [0, 1]}
     ]

    var colors = d3.scale.category10();
    console.log(colors);
    desc.forEach(function (d, i) {
      d.color = colors('' + i);
    });
    
    const p = new LineUpJS.provider.LocalDataProvider(arr, desc);    
     {
      const r = p.pushRanking();
         r.push(p.create(desc[0]));
          r.push((function () {
        const rstack = p.create(LineUpJS.model.createStackDesc('Stack'));
        rstack.push(p.create(desc[1]));
        rstack.push(p.create(desc[2]));
     
        rstack.setWeights([1, 1]);
        return rstack;
      })());
    
    }
    const instance = LineUpJS.create(p, document.getElementById('lineuptable'), {
     
    });
    instance.update();
});
